In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/big-data-analytics-certification-kr-2023-5th/sample_submission.csv
/kaggle/input/big-data-analytics-certification-kr-2023-5th/train.csv
/kaggle/input/big-data-analytics-certification-kr-2023-5th/test.csv


# 0. 데이터 로드

In [2]:
import pandas as pd
train = pd.read_csv("/kaggle/input/big-data-analytics-certification-kr-2023-5th/train.csv")
test = pd.read_csv("/kaggle/input/big-data-analytics-certification-kr-2023-5th/test.csv")

print(train.shape) # 행, 열 크기 확인
display(train.head())

print(test.shape) # 행, 열 크기 확인
display(test.head())

(3759, 9)


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,EcoSport,2017,10298,Manual,25013,Petrol,150,53.3,1.0
1,Focus,2016,10491,Manual,30970,Diesel,0,74.3,1.5
2,S-MAX,2017,13498,Manual,60200,Diesel,150,56.5,2.0
3,Fiesta,2018,10600,Manual,27380,Petrol,145,56.5,1.0
4,Fiesta,2018,11000,Manual,7724,Petrol,145,65.7,1.0


(1617, 8)


,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,S-MAX,2016,Manual,23905,Diesel,125,56.5,2.0
1,Fiesta,2018,Manual,16895,Petrol,145,40.3,1.5
2,Kuga,2017,Manual,12109,Petrol,165,45.6,1.5
3,Kuga,2018,Semi-Auto,13940,Diesel,145,58.9,1.5
4,Fiesta,2013,Manual,31690,Petrol,30,54.3,1.2


# 1. EDA

In [3]:
# train EDA

print(train.shape) # 행, 열 크기 확인
print()
print(train.info()) # 데이터 타입 확인
print()
print(train.isnull().sum()) # 결측치 확인
display(train.describe()) # 기초통계량

display(train.describe(include="O")) # 범주형 변수 unique 확인

(3759, 9)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3759 entries, 0 to 3758
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3759 non-null   object 
 1   year          3759 non-null   int64  
 2   price         3759 non-null   int64  
 3   transmission  3759 non-null   object 
 4   mileage       3759 non-null   int64  
 5   fuelType      3759 non-null   object 
 6   tax           3759 non-null   int64  
 7   mpg           3759 non-null   float64
 8   engineSize    3759 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 264.4+ KB
None

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64


,year,price,mileage,tax,mpg,engineSize
count,3759.000000,3759.000000,3759.000000,3759.000000,3759.000000,3759.000000
mean,2016.887736,12353.321362,23191.786379,114.444001,57.793083,1.352807
std,2.151811,4750.338051,19223.258713,61.713682,10.099221,0.437130
min,2002.000000,1095.000000,5.000000,0.000000,20.900000,0.000000
25%,2016.000000,9000.000000,9997.500000,75.000000,52.300000,1.000000
50%,2017.000000,11495.000000,18180.000000,145.000000,58.900000,1.200000
75%,2018.000000,15310.000000,31000.000000,145.000000,65.700000,1.500000
max,2060.000000,48000.000000,144000.000000,570.000000,201.800000,5.000000


,model,transmission,fuelType
count,3759,3759,3759
unique,19,3,3
top,Fiesta,Manual,Petrol
freq,1372,3234,2541


In [4]:
# year의 max값이 2060이라 칼럼의 값 확인
train["year"].value_counts()

2017    1051
2018     871
2019     656
2016     472
2015     238
2014     178
2013     130
2020      54
2011      24
2010      22
2009      17
2012      17
2008      12
2007       7
2005       3
2006       3
2060       1
2002       1
2003       1
2004       1
Name: year, dtype: int64

- 중고 데이터임을 감안한다면 2060년은 미래의 값이라 말이 안되는 수치
- year 칼럼에서 2060은 잘못 기입된 것 (이상치)로 판단되어 제거하는게 옳다고 판단
- 2060과 같은 값 혹은 유사한 값이 많았다면 제거가 아닌 대체 방법을 고려하는 방향이 적절하나, 여기서는 2060인 데이터가 하나이므로 삭제해도 큰 영향이 없을 것으로 판단 

In [5]:
# train 데이터프레임에서 year 값이 2060인 행을 제거하는 코드

train = train[train["year"] != 2060]
len(train) # 원래 행 : 3759 -> 하나 줄어야함

3758

In [6]:
# test EDA

print(test.info()) # 데이터 타입 확인
print()
print(test.isnull().sum())
display(test.describe()) # 기초통계량
display(test.describe(include="O")) # 범주형 변수 unique 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1617 entries, 0 to 1616
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1617 non-null   object 
 1   year          1617 non-null   int64  
 2   transmission  1617 non-null   object 
 3   mileage       1617 non-null   int64  
 4   fuelType      1617 non-null   object 
 5   tax           1617 non-null   int64  
 6   mpg           1617 non-null   float64
 7   engineSize    1617 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 101.2+ KB
None

model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64


,year,mileage,tax,mpg,engineSize
count,1617.000000,1617.000000,1617.000000,1617.000000,1617.000000
mean,2016.752628,23817.544836,113.089054,57.804082,1.362894
std,2.182152,20130.026364,63.584868,10.567025,0.436878
min,2005.000000,5.000000,0.000000,20.900000,0.000000
25%,2016.000000,10422.000000,30.000000,52.300000,1.000000
50%,2017.000000,18471.000000,145.000000,58.900000,1.200000
75%,2018.000000,31098.000000,145.000000,65.700000,1.600000
max,2020.000000,150890.000000,570.000000,201.800000,5.000000


,model,transmission,fuelType
count,1617,1617,1617
unique,19,3,3
top,Fiesta,Manual,Petrol
freq,591,1385,1085


- train, test 모두 범주형 변수의 unique값 동일, 별다른 이상이 없어보여서 다음 과정 진행

# 2. 데이터 전처리

In [7]:
# 예측할 값은 price이므로 y로 할당해주기 (train에만 있는 값 !)
# pop을 이용해서 X, y 아예 분리

y = train.pop("price")

display(y.head())
display(train.head()) # 잘 분리됐는지 확인

0    10298
1    10491
2    13498
3    10600
4    11000
Name: price, dtype: int64

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,EcoSport,2017,Manual,25013,Petrol,150,53.3,1.0
1,Focus,2016,Manual,30970,Diesel,0,74.3,1.5
2,S-MAX,2017,Manual,60200,Diesel,150,56.5,2.0
3,Fiesta,2018,Manual,27380,Petrol,145,56.5,1.0
4,Fiesta,2018,Manual,7724,Petrol,145,65.7,1.0


## 수치형 변수 스케일링
- 중요 : train 데이터에만 fit 해준다. 
    - 즉, train에는 fit_transform, test에는 transform

In [8]:
from sklearn.preprocessing import StandardScaler

# cols = ["", "", "", ...] # 수치형 칼럼명 직접 지정
# con_cols = train.select_dtypes(exclude = "object").copy

con_df = train.select_dtypes(include=['float64', 'int64']).copy() # 수치형 칼럼명 가져오기
con_cols = con_df.columns


display("train 스케일링 전", train.head()) # 스케일링 전 확인
display("test 스케일링 전", test.head()) # 스케일링 전 확인

scaler = StandardScaler()

train[con_cols] = scaler.fit_transform(train[con_cols])
test[con_cols] = scaler.transform(test[con_cols])

display("train 스케일링 후", train.head()) # 스케일링 후 확인
display("test 스케일링 후", test.head()) # 스케일링 후 확인

'train 스케일링 전'

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,EcoSport,2017,Manual,25013,Petrol,150,53.3,1.0
1,Focus,2016,Manual,30970,Diesel,0,74.3,1.5
2,S-MAX,2017,Manual,60200,Diesel,150,56.5,2.0
3,Fiesta,2018,Manual,27380,Petrol,145,56.5,1.0
4,Fiesta,2018,Manual,7724,Petrol,145,65.7,1.0


'test 스케일링 전'

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,S-MAX,2016,Manual,23905,Diesel,125,56.5,2.0
1,Fiesta,2018,Manual,16895,Petrol,145,40.3,1.5
2,Kuga,2017,Manual,12109,Petrol,165,45.6,1.5
3,Kuga,2018,Semi-Auto,13940,Diesel,145,58.9,1.5
4,Fiesta,2013,Manual,31690,Petrol,30,54.3,1.2


'train 스케일링 후'

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,EcoSport,0.060846,Manual,0.095212,Petrol,0.576700,-0.445420,-0.807071
1,Focus,-0.430891,Manual,0.405209,Diesel,-1.854576,1.634559,0.336756
2,S-MAX,0.060846,Manual,1.926310,Diesel,0.576700,-0.128471,1.480583
3,Fiesta,0.552582,Manual,0.218388,Petrol,0.495658,-0.128471,-0.807071
4,Fiesta,0.552582,Manual,-0.804491,Petrol,0.495658,0.782758,-0.807071


'test 스케일링 후'

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,S-MAX,-0.430891,Manual,0.037553,Diesel,0.171488,-0.128471,1.480583
1,Fiesta,0.552582,Manual,-0.327241,Petrol,0.495658,-1.733025,0.336756
2,Kuga,0.060846,Manual,-0.576300,Petrol,0.819828,-1.208078,0.336756
3,Kuga,0.552582,Semi-Auto,-0.481016,Diesel,0.495658,0.109241,0.336756
4,Fiesta,-1.906100,Manual,0.442677,Petrol,-1.368321,-0.346373,-0.349540


## 범주형 변수 인코딩
- 선형회귀는 원-핫 인코딩을 적용해야함
    - 선형회귀에 라벨 인코더 적용하면 문제 발생 (회귀 계수 연산 시, 라벨 인코딩 된 후 숫자의 크고 작음이 모델에 부정적 영향 끼칠 가능성)

In [9]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

print(train.shape, test.shape)

(3758, 30) (1617, 30)


# 3. 검증 데이터 분리

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y, test_size = 0.2, random_state = 2023)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(3006, 30) (752, 30) (3006,) (752,)


# 4. 회귀 모델링 및 비교
- 회귀에 많이 쓰는 랜포, xgboost, lgbm 비교
- RMSE 값이 작을수록 모델의 성능이 좋다고 평가

In [11]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [12]:
# rmse
from sklearn.metrics import mean_squared_error
import numpy as np

def rmse(y_true, y_pred) :
    return np.sqrt(mean_squared_error(y_true, y_pred))

## 랜덤포레스트

In [13]:
rf = RandomForestRegressor(random_state=2023)
rf.fit(X_train, y_train)
rf.pred = rf.predict(X_val) # 검증용 X_val
rmse(y_val, rf.pred)

1285.3688233845553

## XGBRegressor

In [14]:
xgb = XGBRegressor(random_state=2023)
xgb.fit(X_train, y_train)
xgb.pred = xgb.predict(X_val) # 검증용 X_val
rmse(y_val, xgb.pred)

1240.644061710228

## LGBMRegressor

In [15]:
lgbm = LGBMRegressor(random_state=2023)
lgbm.fit(X_train, y_train)
lgbm.pred = lgbm.predict(X_val) # 검증용 X_val
rmse(y_val, lgbm.pred)

1292.41905178064

### 최종 모델로 xgb 선택

# 5. 제출 : df, csv

In [16]:
submission = pd.read_csv('/kaggle/input/big-data-analytics-certification-kr-2023-5th/sample_submission.csv')
print(submission.head())
print(submission.shape)

   id  price
0   0      0
1   1      0
2   2      0
3   3      0
4   4      0
(1617, 2)


In [17]:
pred = xgb.predict(test) # 검증용 X_val
submission['price'] = pred
submission.head()

,id,price
0,0,15627.256836
1,1,16626.052734
2,2,14513.416992
3,3,18318.572266
4,4,6235.977051


In [18]:
submission.to_csv("submission.csv", index=False)

# 결과확인
# pd.read_csv('submission.csv') 